In [1]:
import datetime
import json
import plotly

from pymongo import MongoClient
from urllib.parse import quote
from plotly.graph_objs import *

In [2]:
plotly.__version__

'4.8.0'

In [3]:
#查詢MongoDB資料庫，去除重複，進行排序，x軸為日期，y軸為每一天的文章篇數
keyword = ''
client = MongoClient('mongodb://127.0.0.1:27017')
db_name = 'appledailytw'
db = client[db_name]

try:
    start_time = '2020-05-27 00:00:00'
    end_time = '2020-05-29 23:59:59'
    #query result
    #db.getCollection('post').find({status_published:{$gt: "2017-10-01 00:00:00",$lt: "2017-11-01 23:59:59"},status_message:{'$regex':''}})
    result = db['post'].find( { 'status_message': { '$regex': keyword },'status_published': {'$gt': start_time, '$lt': end_time}    }).sort("_id", -1)
    print ("Querying in database: %s" %(db_name))
    print ("total: %s records found." % result.count())

    result = list(result)
    temp_dict = {}
    date_dict = {}
    for x in result:
        temp_dict[ x['status_id'] ] = x
    print ("distinct %s records found." % (len(temp_dict) ) )

    post_dict = {}
    post_counter = 0
    last_sum = 0
    for x in temp_dict:
        last_sum = 0
        post_counter = 0
        #dt = temp_dict[x]["status_published"].strftime("%Y-%m-%d")
        dt = temp_dict[x]["status_published"][:10]
        try:
            last_sum = post_dict[dt]['counter']
            post_dict[dt]={'counter':last_sum+1}
        except Exception as e:
            post_counter = post_counter+1
            post_dict[dt]={'counter':post_counter}
            pass
    x_axis1 = []
    y_axis1 = []
    for x in post_dict:
        x_axis1.append(x)
        y_axis1.append(post_dict[x]['counter'])
    zipped = zip(x_axis1, y_axis1)
    x_axis1 = []
    y_axis1 = []
    zipped = sorted(zipped, key=lambda x: x[0])
    for x in zipped:
        x_axis1.append(x[0])
        y_axis1.append(x[1])
    print(x_axis1)
    print(y_axis1)
except Exception as e:
    print('Exception:'+str(e))
    pass


Querying in database: appledailytw
total: 0 records found.
distinct 0 records found.
[]
[]


C:\Users\u0151051\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


In [4]:
#繪圖"
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({
        "data": [{
                "x": x_axis1,
                "y": y_axis1
            }],
        "layout": {
            "title": "David 蘋果日報FB 每日發文數統計"
        }
    })

In [5]:
#對已經查完的資料Dictionary temp_dic，x軸為日期，y軸互動數累計
for x in temp_dict:
    last_sum = 0
    post_counter = 0
    #dt = temp_dict[x]["status_published"].strftime("%Y-%m-%d")
    dt = temp_dict[x]["status_published"][:10]
    try:
        last_sum = post_dict[dt]['num_reactions']
        post_dict[dt]={'num_reactions':last_sum+temp_dict[x]['num_reactions']}
    except Exception as e:
        post_counter = post_counter+1
        post_dict[dt]={'num_reactions':post_counter}
        pass
#print(date_dict)
x_axis = []
y_axis = []
for x in post_dict:
    x_axis.append(x)
    y_axis.append(post_dict[x]['num_reactions'])

zipped = zip(x_axis, y_axis)
x_axis2 = []
y_axis2 = []
zipped = sorted(zipped, key=lambda x: x[0])
for x in zipped:
    x_axis2.append(x[0])
    y_axis2.append(x[1])
print(x_axis2)
print(y_axis2)

[]
[]


In [6]:
#繪圖
plotly.offline.init_notebook_mode(connected=True)
plotly.offline.iplot({
        "data": [{
                "x": x_axis2,
                "y": y_axis2
            }],
        "layout": {
            "title": "David 蘋果日報FB 每日按讚數統計"
        }
    })

In [7]:
#繪圖, y_axis1為發每日文統計，y_axis2為每日按讚數，錯誤的堆疊比較
plotly.offline.init_notebook_mode(connected=True)
trace0 = Scatter(
    x=x_axis1,
    y=y_axis1,
    name = '每日發文篇數',
    line = dict(
        color = ('rgb(244, 67, 54)'),
        width = 4)
)
trace1 = Scatter(
    x=x_axis2,
    y=y_axis2,
    name = '每日按讚數',
    line = dict(
        color = ('rgb(33, 150, 243)'),
        width = 4)
)
data = Data([trace0, trace1])
plotly.offline.iplot({
        "data": data,
        "layout": {
            "title": "David 蘋果日報FB 每日發文/按讚數統計"
        }
    })

C:\Users\u0151051\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:40: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




In [8]:
#整理分享
for x in temp_dict:
    last_sum = 0
    post_counter = 0
    #dt = temp_dict[x]["status_published"].strftime("%Y-%m-%d")
    dt = temp_dict[x]["status_published"][:10]
    try:
        last_sum = post_dict[dt]['num_shares']
        post_dict[dt]={'num_shares':last_sum+temp_dict[x]['num_shares']}
    except Exception as e:
        post_counter = post_counter+1
        post_dict[dt]={'num_shares':post_counter}
        pass
#print(date_dict)
x_axis = []
y_axis = []
for x in post_dict:
    x_axis.append(x)
    y_axis.append(post_dict[x]['num_shares'])

zipped = zip(x_axis, y_axis)
x_axis3 = []
y_axis3 = []
zipped = sorted(zipped, key=lambda x: x[0])
for x in zipped:
    x_axis3.append(x[0])
    y_axis3.append(x[1])
print(x_axis3)
print(y_axis3)

[]
[]


In [9]:
#整理留言
for x in temp_dict:
    last_sum = 0
    post_counter = 0
    #dt = temp_dict[x]["status_published"].strftime("%Y-%m-%d")
    dt = temp_dict[x]["status_published"][:10]
    try:
        last_sum = post_dict[dt]['num_comments']
        post_dict[dt]={'num_comments':last_sum+temp_dict[x]['num_comments']}
    except Exception as e:
        post_counter = post_counter+1
        post_dict[dt]={'num_comments':post_counter}
        pass
#print(date_dict)
x_axis = []
y_axis = []
for x in post_dict:
    x_axis.append(x)
    y_axis.append(post_dict[x]['num_comments'])

zipped = zip(x_axis, y_axis)
x_axis4 = []
y_axis4 = []
zipped = sorted(zipped, key=lambda x: x[0])
for x in zipped:
    x_axis4.append(x[0])
    y_axis4.append(x[1])
print(x_axis4)
print(y_axis4)

[]
[]


In [10]:
#繪圖, 每日分享數、每日留言數，兩條趨勢圖
plotly.offline.init_notebook_mode(connected=True)
trace0 = Scatter(
    x=x_axis3,
    y=y_axis3,
    name = '每日分享數',
    line = dict(
        color = ('rgb(76, 175, 80)'),
        width = 4)
)
trace1 = Scatter(
    x=x_axis4,
    y=y_axis4,
    name = '每日留言數',
    line = dict(
        color = ('rgb(255, 87, 34)'),
        width = 4)
)
data = Data([trace0, trace1])
plotly.offline.iplot({
        "data": data,
        "layout": {
            "title": "David 蘋果日報FB 每日分享/留言數統計"
        }
    })

In [11]:
# 回家作業
# 1. 請選擇你欲觀察的Facebook粉絲頁，並進行爬蟲撰寫
# 2. 請將爬蟲爬下來的粉絲頁文章資料，寫入本地的MongoDB資料庫
# 3. 開啟jupyter notebook，接續先前的範例，彙整按讚、留言、分享分析為三條曲線於同一張圖上

# 其他提示
# a. 粉絲頁：自己尋找
# b. 觀察日期區間：自己定義

# 報告
# Q:因為你的爬蟲，你對該粉絲頁的insight?